In [ ]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
torch.manual_seed(0)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

from torchvision.datasets import EMNIST

transform = transforms.Compose([transforms.ToTensor()])

train_dataset = EMNIST(root='data', split='letters', train=True, transform=transform, download=True)
test_dataset = EMNIST(root='data', split='letters', train=False, transform=transform, download=True)

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=200, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=2000, shuffle=False)

input_channels = 1  # Grayscale images
num_classes = 814  # Number of classes in EMNIST ByClass

model = nn.Sequential(
    nn.Conv2d(input_channels, 32, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.Conv2d(32, 64, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Flatten(),
    nn.Linear(64 * 14 * 14, 128),
    nn.ReLU(),
    nn.Linear(128, num_classes)
).to(device)

opt = torch.optim.Adam(model.parameters(), lr=0.001)


def train():
    for images, labels in train_loader:
        # Move tensors to the configured device, and convert image to vector.
        images = images.to(device)
        labels = labels.to(device)

        # Forward pass
        logits = model(images).squeeze((-1, -2)) #output shape is [N, 10]

        # Backpropagation and optimization
        loss = nn.functional.cross_entropy(logits, labels)
        loss.backward()
        opt.step()
        opt.zero_grad()

def test(epoch):
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            #Convert image pixels to vector
            images = images.to(device)
            labels = labels.to(device)

            # Forward pass
            logits = model(images).squeeze((-1, -2))

            # Compute total correct so far
            predicted = torch.argmax(logits, -1)
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
        print(f'Test accuracy after {epoch+1} epochs: {100 * correct / total} %')


# Run training
for epoch in range(int(1)):
    train()
    test(epoch)




In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = '1'


In [ ]:
import random

def display_random_image_prediction():
    with torch.no_grad():
        # Choose a random index from the EMNIST test dataset
        index_to_display = random.randint(0, len(test_dataset) - 1)

        # Extract the image and label from the EMNIST test dataset
        image, label = test_dataset[index_to_display]
        image = image.unsqueeze(0).to(device)  # Add batch dimension and move to device

        # Forward pass
        logits = model(image).squeeze((-1, -2))
        predicted_label = torch.argmax(logits).item()

        # Display the image
        image = image.squeeze().cpu().numpy()
        plt.imshow(image, cmap='gray')
        plt.title(f"Actual: {chr(label + 96)} (Class {label}), Predicted: {chr(predicted_label + 96)} (Class {predicted_label})")
        plt.show()

# Display the image and its predicted result
display_random_image_prediction()


NameError: name 'torch' is not defined